<a href="https://colab.research.google.com/github/starirene9/Algorithm-Udemy/blob/master/%5B2%EC%A1%B0%5D_%5B4%EC%A3%BC%EC%B0%A8%5D_STT_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📊 Face Detection + Sticker App Refactoring & STT/TTS Feature Enhancement Report

---

1. Structural Improvement: Functional → Class-Based

- Structure
  Before: Used global functions and variables
  After: Introduced `FaceStickerApp` class to encapsulate state and functionalities

- Advantages
  Before: Quick to run
  After:
    - Improved state management
    - Easier to understand
    - Enhanced reusability and scalability
    - Easier to test

- Key Improvement Point
  Before: Difficult to scale due to global variable usage
  After: Achieved modularity and clear responsibility separation through class-based refactoring

---

2. STT / TTS Feature Additions

- STT (Speech to Text)
  Before: Not available
  After: Added `speech_to_text()` to automatically convert voice input to text

- TTS (Text to Speech)
  Before: Not available
  After: Added `generate_tts_file()` to generate voice output from guidance text

- Visual + Audio Feedback
  Before: Not available
  After: Displays STT-recognized text at the top-left and plays TTS audio

- Improvement Effects
  - Enhanced user feedback by combining voice and visual information
  - Improved accessibility for visually impaired and elderly users

---

3. Key Points of Code Refactoring

- Removed duplicated logic by separating into functions: `draw_face_number`, `is_dark_background`, `save_image`
- Dynamically generated text input fields for up to 10 faces
- Modularized CSS for sticker buttons using `generate_sticker_css()`
- Unified image saving process using the `save_image()` function

---

4. UI / UX Enhancements

- Layout
  Before: Single column layout
  After: Tabbed structure for better separation of upload and result views

- Text Input UI
  Before: Not available
  After: Dynamically shows text fields only for selected faces

- User Guide
  Before: Simple text listing
  After: Step-by-step instructions for better clarity

- Responsive Design
  Before: Basic default styling
  After: Improved mobile support and cleaner CSS styling

---

5. Feature Integration & Scalability Enhancement

- Integrated sticker, text, and voice features into one flow using `apply_sticker_dynamic()`
- Text color auto-adjusts based on background brightness for readability
- Centralized management of `show_dict` and `overlay_texts_dict` inside the class to strengthen inter-feature linkage

---

Future Improvement Suggestions

- Add drag-based face selection UI
- Add STT text editing feature
- Add user options for TTS voice style and speed
- Implement automatic face grouping and template generation
---


In [1]:
!pip install git+https://github.com/ultralytics/ultralytics.git@main
!pip install gradio
# TTS (gTTS) + STT (SpeechRecognition) 설치
!pip install gTTS SpeechRecognition pydub
!pip install gradio ultralytics speechrecognition pydub gTTS opencv-python pillow

# base model download : wget 웹(URL)에서 파일을 가져올 때 사용
!wget https://github.com/akanametov/yolo-face/releases/download/v0.0.0/yolov11n-face.pt
# !wget https://github.com/akanametov/yolo-face/releases/download/v0.0.0/yolov11s-face.pt

# 스티커 이미지 download : gdown Google Drive에서 파일을 다운로드
!gdown https://drive.google.com/uc?id=1vS5pgJErqRODIz60HfJV1FIdbYlllK93
!gdown https://drive.google.com/uc?id=1ZgtBE43SDqrljluG_UZVhsuPinDChQUS
!gdown https://drive.google.com/uc?id=1RfwLdfhT5XwKT0QRTNrMH0ESH5ujjxC3
!gdown https://drive.google.com/uc?id=1hPkuLWObT7JID1BxFEXfkzve6TTAe6dT
!gdown https://drive.google.com/uc?id=1vnbsO_0rAkLKPZZUVM2DSLZ9yta1ZOAJ


# font download : apt-get 터미널에서 소프트웨어(패키지)**를 설치하거나 삭제, 업데이트할 때 사용
!apt-get install -y fonts-dejavu
!apt-get install -y fonts-noto-cjk
!apt-get install -y ffmpeg


  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-ahr2dxe6
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-ahr2dxe6
  Resolved https://github.com/ultralytics/ultralytics.git to commit fbbe463d3c2539211fbeda77d75da3bda4097e3f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import base64
import gradio as gr
from ultralytics import YOLO
import PIL.Image as Image
from PIL import ImageDraw, ImageFont
import io
from gtts import gTTS  # TTS
import tempfile
import speech_recognition as sr  # STT

# 클래스 기반으로 구조를 변경하여, 전역 상태와 기능을 하나의 객체로 관리할 수 있게 함.
class FaceStickerApp:
    def __init__(self):
        # 모델 및 폰트 초기화 (이전에는 글로벌 변수였음)
        self.model = YOLO("/content/yolov11n-face.pt")
        self.font = ImageFont.truetype("/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc", 30)

        # 스티커 옵션 정의 (이전에는 전역 상수였음(## 상수 변경))
        self.STICKER_OPTIONS = {
            "Sticker 1": "/content/12870971.png",
            "Sticker 2": "/content/3508689.png",
            "Sticker 3": "/content/heart.png",
            "Sticker 4": "/content/nature.png",
            "Sticker 5": "/content/star.png"
        }
        # 얼굴 검출 결과 및 텍스트 오버레이 정보를 클래스 속성으로 관리
        self.show_dict = {}           # {face_index: {'xyxy': [x1, y1, x2, y2], 'conf': confidence}}
        self.overlay_texts_dict = {}  # {face_index: text}

    def predict_faces(self, img, conf_threshold, iou_threshold):
        """
        이미지에서 얼굴을 감지하고, 얼굴 바운딩 박스와 번호를 그립니다.
        리팩토링: 기존 함수에서 전역 변수를 사용하던 것을 클래스 속성(self.show_dict)로 대체.
        """
        copy_image = img.copy().convert('RGB')
        results = self.model.predict(source=copy_image, conf=conf_threshold, iou=iou_threshold)[0]
        self.show_dict = {}
        draw = ImageDraw.Draw(copy_image)

        for i, r in enumerate(results):
            xyxy = r.boxes.xyxy.tolist()[0]
            confs = r.boxes.conf.tolist()[0]
            # cls가 0이면 얼굴로 판단
            if r.boxes.cls.int() == 0:
                self.show_dict[i] = {'xyxy': xyxy, 'conf': confs}
                x1, y1, x2, y2 = xyxy
                draw.rectangle([x1, y1, x2, y2], outline="red", width=6)
                self.draw_face_number(draw, i, x1, y1)
        return copy_image

    def draw_face_number(self, draw, index, x1, y1):
        """
        얼굴 바운딩 박스 위에 얼굴 번호를 표시합니다.
        리팩토링: 번호 표시 기능을 별도 함수로 분리하여 재사용성을 높임.
        """
        text = f"{index+1}"
        try:
            bbox = self.font.getbbox(text)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]
            text_x, text_y = x1, y1 - text_height - 3
            # 검은색 배경 사각형을 그려 텍스트 가독성을 높임
            draw.rectangle([text_x, text_y, text_x + text_width + 8, text_y + text_height + 4], fill="black")
            draw.text((text_x + 4, text_y + 2), text, fill="white", font=self.font)
        except AttributeError:
            # PIL 버전에 따른 예외 처리
            draw.text((x1, y1 - 45), text, fill="white", font=self.font, stroke_width=2, stroke_fill="black")

    def speech_to_text(self, audio):
        """
        오디오 파일을 받아 Google Speech Recognition을 통해 텍스트로 변환합니다.
        리팩토링: 기능별 함수를 클래스 메서드로 통합하여 코드 일관성 향상.
        """
        if not audio or not isinstance(audio, str):
            return "유효한 오디오 파일이 제공되지 않았습니다."
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio) as source: # 오디오 객체를 source로 참조함
            audio_data = recognizer.record(source)
            try:
                return recognizer.recognize_google(audio_data, language="ko-KR")
            except sr.UnknownValueError:
                return "음성을 인식하지 못했습니다."
            except sr.RequestError:
                return "STT 요청 실패."


    def generate_tts_file(self, text):
        if not text.strip():
            return None
        tts = gTTS(text=text.strip(), lang='ko')
        output_path = "/tmp/title_sticker_announce.mp3"
        tts.save(output_path)
        return output_path

    def is_dark_background(self, img, x1, y1, x2, y2):
        """
        이미지의 특정 영역이 어두운지 판단합니다.
        """
        crop = img.crop((x1, y1, x2, y2)).convert("L") # img.crop() 특정 영역 잘라서 (L) 흑백으로 변환
        hist = crop.histogram() # 픽셀 밝기값의 분포
        pixels = sum(hist) # 전체 픽셀 수
        brightness = sum(i * hist[i] for i in range(256)) / pixels # 밝기값 평균 = (밝기 총합) ÷ (전체 픽셀 수)
        return brightness < 200

    def update_text_inputs(self, selected_faces):
        """
        선택된 얼굴 번호에 따라 동적 텍스트 입력 필드를 업데이트합니다.
        리팩토링: 최대 10개 필드를 for 루프로 생성하여 중복 코드를 줄임.
        """
        selected_faces = sorted(selected_faces, key=lambda x: int(x)) if selected_faces else []
        max_count = 10
        updates = []
        for i in range(max_count):
            if i < len(selected_faces):
                face_num = selected_faces[i]
                current_text = self.overlay_texts_dict.get(int(face_num) - 1, "")
                updates.append(gr.update(visible=True, label=f"Text for Face {face_num}", value=current_text))
            else:
                updates.append(gr.update(visible=False, value=""))
        return updates

    """
    dict.get(key, 기본값)
    	•	딕셔너리에서 key가 있으면 → 해당 value 반환
	    •	key가 없으면 → 기본값 반환 (여기선 "")
    """

    def apply_sticker_dynamic(self, img, selected_boxes, conf_threshold, iou_threshold, transcribed_text,
                                text1, text2, text3, text4, text5, text6, text7, text8, text9, text10, sticker_name):
        """
        선택된 얼굴에는 입력 텍스트를 오버레이하고, 선택되지 않은 얼굴에는 스티커를 적용합니다.
        또한, STT로 받은 텍스트를 이미지 왼쪽 상단에 오버레이합니다.
        리팩토링:
          - 스티커 적용과 텍스트 오버레이 기능을 하나의 함수로 통합.
          - STT 텍스트 처리 부분을 추가하여, 사용자 피드백을 이미지에 바로 표시.
        """
        # 선택한 스티커 로드
        sticker_path = self.STICKER_OPTIONS[sticker_name]
        sticker = Image.open(sticker_path)
        draw = ImageDraw.Draw(img)

        # 선택된 얼굴 번호(1-indexed)를 0-indexed로 변환 후 정렬
        adjusted_boxes = sorted([int(x) - 1 for x in selected_boxes])

        # 텍스트 입력 필드에서 빈 값은 제외한 리스트 구성
        input_texts = [t for t in [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10] if t]

        """
        [실행할 코드 for 변수 in 반복할 것들 if 조건]

        이건 이렇게 해석돼:
        •	t: 반복하면서 꺼낸 값 (텍스트 하나씩)
        •	for t in [...10개 텍스트...]: 10개의 텍스트 중에서 하나씩 꺼냄
        •	if t: 값이 비어있지 않은 경우에만
        •	[...]: 그 t를 결과 리스트에 추가함
        """

        # 선택된 얼굴 순서대로 오버레이 텍스트 업데이트
        for idx, face in enumerate(adjusted_boxes): # face: 현재 반복 중인 얼굴 번호
            if idx < len(input_texts):
                self.overlay_texts_dict[face] = input_texts[idx]

        # 각 검출된 얼굴에 대해 처리
        for i in self.show_dict.keys():
            x1, y1, x2, y2 = map(int, self.show_dict[i]['xyxy'])
            width, height = x2 - x1, y2 - y1
            if i not in adjusted_boxes:
                # 선택되지 않은 얼굴에는 스티커 적용
                sticker_resized = sticker.resize((width, height))
                img.paste(sticker_resized, (x1, y1), sticker_resized)
            else:
                # 선택된 얼굴에는 텍스트 오버레이
                text_to_overlay = self.overlay_texts_dict.get(i, "")
                if text_to_overlay:
                    text_bbox = self.font.getbbox(text_to_overlay)
                    text_width = text_bbox[2] - text_bbox[0]
                    text_height = text_bbox[3] - text_bbox[1]
                    text_x = x1 + (width - text_width) // 2
                    text_y = y2 + 5
                    if self.is_dark_background(img, x1, y1, int(x1+text_width), int(y1+text_height)):
                        draw.text((text_x, text_y), text_to_overlay, fill="black", font=self.font, stroke_width=2, stroke_fill="white")
                    else:
                        draw.text((text_x, text_y), text_to_overlay, fill="black", font=self.font)

        # STT 결과 텍스트를 이미지 왼쪽 상단에 오버레이 (가독성을 위해 배경 사각형 추가)
        if transcribed_text:
            margin = 10
            top_left_bbox = self.font.getbbox(transcribed_text)
            text_width = top_left_bbox[2] - top_left_bbox[0]
            text_height = top_left_bbox[3] - top_left_bbox[1]
            if self.is_dark_background(img, 0, 0, int(text_width+5), int(text_height+5)):
                draw.text((margin + 4, margin + 2), transcribed_text, fill="black", font=self.font, stroke_width=2, stroke_fill="white")
            else:
                draw.text((margin + 4, margin + 2), transcribed_text, fill="black", font=self.font)
            tts_text = f"말씀하신 {transcribed_text} 가 title 스티커로 좌측 상단에 부착되었습니다."
        else:
            tts_text = "STT 텍스트 또는 수동 입력이 비어있습니다."
        tts_file_path = self.generate_tts_file(tts_text)

        return img, gr.update(visible=True), tts_file_path

    def save_image(self, img, format="PNG"):
        """
        이미지를 지정 경로에 저장하고 경로를 반환합니다.
        리팩토링: 기존 단순 저장 로직을 함수로 분리하여 재사용성을 확보.
        """
        file_path = "/tmp/final_image.png"
        img.save(file_path, format=format)
        return file_path

    def select_sticker(self, name, sticker_buttons_list):
        """
        스티커 버튼 선택 시 UI 업데이트 (선택된 버튼 강조)
        리팩토링: UI 업데이트 로직을 별도 함수로 분리하여, 여러 곳에서 재사용 가능하게 함.
        """
        button_updates = []
        for btn_name in sticker_buttons_list:
            if btn_name == name:
                button_updates.append(gr.update(elem_classes=["selected-button"]))
            else:
                button_updates.append(gr.update(elem_classes=[]))
        return [name, f"**{name}** selected."] + button_updates

    def to_base64(self, img_path):
        """
        이미지 파일을 base64 문자열로 변환하여 CSS에 사용합니다.
        """
        if not os.path.exists(img_path):
            return ""
        with open(img_path, "rb") as f:
            data = f.read()
        return "data:image/png;base64," + base64.b64encode(data).decode("utf-8")

    def generate_sticker_css(self):
        """
        스티커 버튼에 사용할 CSS 스타일을 생성합니다.
        리팩토링: CSS 생성 로직을 함수로 분리하여, UI 구성 코드와 분리.
        """
        sticker_css = ""
        for idx, (sname, spath) in enumerate(self.STICKER_OPTIONS.items(), 1):
            icon_data = self.to_base64(spath)
            sticker_css += f"""
            #sticker{idx} {{
                background-image: url("{icon_data}");
                background-size: 24px 24px;
                background-repeat: no-repeat;
                background-position: 8px center;
                padding-left: 40px;
            }}
            """
        return sticker_css

    def process_and_update(self, img, conf, iou):
        """
        얼굴 검출 후 결과 이미지를 반환하고,
        얼굴 선택 체크박스에 들어갈 번호 목록을 업데이트합니다.
        리팩토링: 검출 결과 처리와 UI 업데이트를 한 함수로 묶어 관리.
        """
        result_img = self.predict_faces(img, conf, iou)
        # 검출된 얼굴마다 오버레이 텍스트 초기화
        self.overlay_texts_dict = {i: "" for i in self.show_dict.keys()}
        bbox_choices = [str(i + 1) for i in self.show_dict.keys()]
        return result_img, gr.update(choices=bbox_choices, value=[]), gr.update(visible=True)

    def save_final_image(self, img):
        """
        최종 결과 이미지를 저장하여 파일 경로를 반환합니다.
        """
        return self.save_image(img, format="PNG")

# 인스턴스 생성 (클래스 기반 리팩토링을 통해 코드의 재사용성과 유지보수성이 향상됨)
app = FaceStickerApp()

# Gradio UI에서 사용할 커스텀 CSS 생성 (스티커 버튼 스타일 포함)
sticker_css = app.generate_sticker_css()
custom_css = f"""
@import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');

.gradio-container h1,
.gradio-container h2,
.gradio-container h3,
.gradio-container .prose p:not(.slider-info-white p):not(.slider-info-white ~ p):not(.slider-info-white + p),
.gradio-container .prose strong,
.gradio-container .prose li,
.gradio-container .markdown-body {{
    color: #000000 !important;
}}

body {{
    margin: 0;
    font-family: 'Roboto', Arial, sans-serif !important;
    background: #ffffff !important;
    padding: 20px;
}}
.gradio-container {{
    max-width: 800px !important;
    margin: auto !important;
    background: #ffffff !important;
    padding: 20px !important;
    border-radius: 10px !important;
    box-shadow: 0 4px 10px rgba(0,0,0,0.1) !important;
}}

.gr-button {{
    background: #008080 !important;
    color: #ffffff !important;
    border: none !important;
    border-radius: 5px !important;
    padding: 10px 20px !important;
    font-size: 16px !important;
    font-weight: 700 !important;
    cursor: pointer !important;
    transition: transform 0.2s, box-shadow 0.2s !important;
}}
.gr-button:hover {{
    transform: translateY(-2px) !important;
    box-shadow: 0 4px 10px rgba(0,0,0,0.2) !important;
}}

.selected-button {{
    box-shadow: 0 4px 10px rgba(0,0,0,0.2) !important;
    background-color: #008080 !important;
}}

.noUi-target {{
    background: #ccc !important;
    border-radius: 3px !important;
}}
.noUi-connect {{
    background: #008080 !important;
    border-radius: 3px !important;
}}
.noUi-handle {{
    background: #008080 !important;
    border: 2px solid #fff !important;
    height: 18px !important;
    width: 18px !important;
    top: -7px !important;
    border-radius: 50% !important;
}}

img {{
    border-radius: 5px !important;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1) !important;
}}

.sticker-container {{
    display: flex !important;
    gap: 10px !important;
    justify-content: center !important;
    flex-wrap: wrap !important;
    margin-bottom: 15px !important;
}}

{sticker_css}

@media (max-width: 600px) {{
    body {{
        font-size: 14px !important;
        padding: 10px !important;
    }}
    .gradio-container {{
        padding: 15px !important;
    }}
    .sticker-container {{
        flex-direction: column !important;
        align-items: center !important;
        margin-bottom: 0 !important;
    }}
    .gr-button {{
        width: 100% !important;
        margin-bottom: 10px !important;
        font-size: 14px !important;
    }}
}}
"""

# Gradio 인터페이스 구성
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    # gr.Markdown("# 얼굴 탐지 및 스티커 적용 애플리케이션")
    # gr.Markdown(
    #     "이미지를 업로드한 뒤 **Detect Faces** 버튼을 누르면, 인식된 얼굴이 표시됩니다. "
    #     "아래 스티커 버튼 중 하나를 선택하고 **Apply Sticker** 버튼을 누르면, "
    #     "선택되지 않은 얼굴에는 스티커가, 선택한 얼굴에는 입력한 텍스트가 오버레이 됩니다."
    # )

    gr.Markdown("## 얼굴 탐지 및 스티커 붙이기(+음성인식 Title 부착 및 결과 음성 안내) 서비스")
    gr.Markdown(
         "이미지를 업로드한 뒤 **Detect Faces** 버튼을 누르면, 인식된 얼굴이 표시됩니다. "
        "아래 스티커 버튼 중 하나를 선택하고 **Apply Sticker** 버튼을 누르면 "
        "선택되지 않은 얼굴에 해당 스티커가 적용됩니다.\n"
        "1) 이미지를 업로드 후 **Detect Faces**로 얼굴 인식.\n"
        "2) **Select Face to Exclude**: 스티커를 붙이지 않을 얼굴 선택.\n"
        "3) 마이크로 말해 **STT** 변환 → 음성으로 인식된 부분은 Title을 생성하여 좌측 상단에 부착.\n"
        "4) Select Face to Exclude에서 **선택한 얼굴 밑에 text 스티커 추가 가능**.\n"
        "5) **Apply Sticker** 버튼으로 최종 이미지 생성.\n"
        "6) **Download Image** 로 다운로드."
    )

    with gr.Column():
        with gr.Tab("Upload Image"):
            image_input = gr.Image(type="pil")
            conf_slider = gr.Slider(minimum=0, maximum=1, value=0.25,
                                    label="Confidence Threshold",
                                    info="얼굴이라고 판단하는 최소 신뢰도.",
                                    elem_classes="slider-info-white")
            iou_slider = gr.Slider(minimum=0, maximum=1, value=0.45,
                                   label="IoU Threshold",
                                   info="검출된 박스들의 중복 병합 임계값.",
                                   elem_classes="slider-info-white")
            detect_button = gr.Button("Detect Faces")
            detected_faces_tab = gr.TabItem("Detected Faces", visible=False)

        with detected_faces_tab:
            result_image = gr.Image(type="pil")
            bbox_selector = gr.CheckboxGroup(label="Select Face(s) for Text Editing", choices=[], type="value")

            # STT 블록: 음성 입력 및 변환 결과 표시
            with gr.Column(elem_id="stt_block") as stt_column:
                gr.Markdown("🎙 **STT - Title에 넣을 텍스트 말하기**")
                with gr.Row():
                    audio_input = gr.Audio(type="filepath", sources=["microphone"])
                    transcribed_text = gr.Textbox(label="Transcribed Text")

            # 동적 텍스트 입력 UI (최대 10개)
            with gr.Column():
                gr.Markdown("✏️ **Enter Text for Each Selected Face**")
                text_inputs = []
                # 반복문으로 동적 텍스트 박스 생성 (리팩토링: 하드코딩된 10개의 컴포넌트를 리스트로 생성)
                for i in range(10):
                    ti = gr.Textbox(label=f"Text for Face {i+1}", visible=False)
                    text_inputs.append(ti)

            # 스티커 선택 정보를 위한 숨겨진 텍스트박스와 선택된 스티커 라벨
            selected_sticker_box = gr.Textbox(value="Sticker 1", visible=False)
            sticker_label = gr.Markdown("**Sticker 1** selected.")

            # 스티커 버튼 생성 (두 줄로 배치)
            sticker_buttons = []
            sticker_names = list(app.STICKER_OPTIONS.keys())
            with gr.Row(elem_classes="sticker-container"):
                for i in range(3):
                    sname = sticker_names[i]
                    btn = gr.Button(sname, elem_id=f"sticker{i+1}", elem_classes=["selected-button"] if sname=="Sticker 1" else [])
                    sticker_buttons.append((btn, sname))
            with gr.Row(elem_classes="sticker-container"):
                for i in range(3, len(sticker_names)):
                    sname = sticker_names[i]
                    btn = gr.Button(sname, elem_id=f"sticker{i+1}")
                    sticker_buttons.append((btn, sname))

            apply_sticker_button = gr.Button("Apply Sticker")
            final_image_tab = gr.TabItem("Final Image", visible=False)

        with final_image_tab:
            final_image = gr.Image(type="pil")
            tts_player = gr.Audio(type="filepath", label="안내 TTS 재생")
            download_button = gr.Button("Download Image")
            download_output = gr.File(label="Download Processed Image")

    # 스티커 버튼 클릭 시 UI 업데이트 (선택된 스티커 강조)
    sticker_buttons_objects = [btn for btn, _ in sticker_buttons]
    for btn, sname in sticker_buttons:
        btn.click(
            fn=lambda sname=sname: app.select_sticker(sname, sticker_names),
            inputs=[],
            outputs=[selected_sticker_box, sticker_label] + sticker_buttons_objects,
            queue=False
        )

    # Detect Faces 버튼 클릭: 얼굴 검출 및 얼굴 선택 옵션 업데이트
    detect_button.click(
        fn=app.process_and_update,
        inputs=[image_input, conf_slider, iou_slider],
        outputs=[result_image, bbox_selector, detected_faces_tab]
    )

    # 오디오 입력 변화 시 STT 처리
    audio_input.change(fn=app.speech_to_text, inputs=[audio_input], outputs=transcribed_text)

    # 얼굴 선택 변경 시 동적 텍스트 입력 UI 업데이트
    bbox_selector.change(
        fn=app.update_text_inputs,
        inputs=bbox_selector,
        outputs=text_inputs
    )

    # Apply Sticker 버튼 클릭: 스티커 및 텍스트 오버레이 적용
    apply_sticker_button.click(
        fn=app.apply_sticker_dynamic,
        inputs=[image_input, bbox_selector, conf_slider, iou_slider, transcribed_text] + text_inputs + [selected_sticker_box],
        outputs=[final_image, final_image_tab, tts_player]
    )

    # apply_sticker_button.click(
    #     fn=apply_sticker_dynamic,
    #     inputs=[image_input, bbox_selector, conf_slider, iou_slider, recognized_text, manual_text, selected_sticker_box],
    #     outputs=[final_image, tts_player]
    # )


    # Download 버튼 클릭: 최종 이미지 저장 및 다운로드
    download_button.click(
        fn=app.save_final_image,
        inputs=final_image,
        outputs=download_output
    )

    # 새로운 이미지 업로드 시 얼굴 선택 옵션 초기화
    image_input.change(
        fn=lambda: gr.update(choices=[], value=[]),
        outputs=bbox_selector
    )

if __name__ == "__main__":
    demo.launch(debug=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1fed79360372902673.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x480 13 faces, 57.5ms
Speed: 46.7ms preprocess, 57.5ms inference, 438.1ms postprocess per image at shape (1, 3, 640, 480)
